# **EDA + Logistic Regression + PCA**


## Tabla de contenido

El contenido de este núcleo se divide en varios temas que son los siguientes:

- La maldición de la dimensionalidad
- Introducción al Análisis de Componentes Principales
- Importar bibliotecas de Python
- Importar conjunto de datos
-	Análisis exploratorio de datos
- Dividir datos en conjunto de entrenamiento y prueba.
- Ingeniería de características
- Escalado de funciones
- Modelo de regresión logística con todas las características.
- Regresión logística con PCA
- Seleccione el número correcto de dimensiones
- Trazar la relación de varianza explicada con el número de dimensiones.
-	Conclusión
- Referencias


## La maldición de la dimensionalidad

Generalmente, los conjuntos de datos del mundo real contienen miles o millones de funciones para entrenar. Esta es una tarea que requiere mucho tiempo ya que hace que el entrenamiento sea extremadamente lento. En estos casos es muy difícil encontrar una buena solución. Este problema a menudo se conoce como la maldición de la dimensionalidad.


**La maldición de la dimensionalidad** se refiere a varios fenómenos que surgen cuando analizamos y organizamos datos en espacios de alta dimensión (a menudo con cientos o miles de dimensiones) que no ocurren en entornos de baja dimensión. El problema es que cuando aumenta la dimensionalidad, el volumen del espacio aumenta tan rápido que los datos disponibles se vuelven escasos. Esta escasez es problemática para cualquier método que requiera significación estadística.


En problemas del mundo real, a menudo es posible reducir considerablemente el número de dimensiones. Este proceso se llama **reducción de dimensionalidad**. Se refiere al proceso de reducir el número de dimensiones consideradas mediante la obtención de un conjunto de variables principales. Ayuda a acelerar el entrenamiento y también es extremadamente útil para la visualización de datos.


La técnica de reducción de dimensionalidad más popular es el Análisis de Componentes Principales (PCA), que se analiza a continuación.



## Introducción al Análisis de Componentes Principales (PCA)


**Análisis de componentes principales (PCA)** es una técnica de reducción de dimensionalidad que se puede utilizar para reducir un conjunto más grande de variables de características a un conjunto más pequeño que aún contiene la mayor parte de la varianza en el conjunto más grande.

### Preservar la variación

PCA primero identifica el hiperplano más cercano a los datos y luego los proyecta en él. Antes de que podamos proyectar el conjunto de entrenamiento en un hiperplano de dimensiones inferiores, debemos seleccionar el hiperplano correcto. La proyección se puede realizar de tal manera que se preserve la varianza máxima. Esta es la idea detrás de PCA.

### Componentes principales

PCA identifica los ejes que representan la cantidad máxima de suma acumulada de varianza en el conjunto de entrenamiento. Estos se denominan componentes principales. PCA supone que el conjunto de datos se centra en el origen. Las clases PCA de Scikit-Learn se encargan de centrar los datos automáticamente.

### Proyectando hasta d Dimensiones

Una vez que hayamos identificado todos los componentes principales, podemos reducir la dimensionalidad del conjunto de datos a d dimensiones proyectándolo en el hiperplano definido por los primeros d componentes principales. Esto garantiza que la proyección conserve la mayor variación posible.

## Import Python libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
import os

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, recall_score, confusion_matrix
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso, ElasticNet
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifierCV, LassoCV, ElasticNetCV

def calculate_metrics(y_true, y_pred_prob, threshold=0.5):
    """
    Calcula las métricas AUC, Gini, F1-score y Recall para un modelo de regresión logística.

    Parameters:
    - y_true: array-like, Ground truth (correct) target values.
    - y_pred_prob: array-like, Predicted probabilities for the positive class.
    - threshold: float, Threshold for converting predicted probabilities to binary predictions.

    Returns:
    - metrics: dict, Dictionary containing AUC, Gini, F1-score and Recall.
    """
    # Calcular las predicciones binarias basadas en el umbral
    y_pred = (y_pred_prob >= threshold).astype(int)

    # Calcular AUC
    auc = roc_auc_score(y_true, y_pred_prob)

    # Calcular Gini
    gini = 2 * auc - 1

    # Calcular F1-score
    f1 = f1_score(y_true, y_pred)

    # Calcular Recall
    recall = recall_score(y_true, y_pred)

    # Guardar métricas en un diccionario
    metrics = {
        'AUC': auc,
        'Gini': gini,
        'F1-score': f1,
        'Recall': recall
    }

    return metrics


def find_best_elasticnet_params(X, y, alphas=np.logspace(-6, 6, 13), l1_ratio=np.linspace(0.01, 1, 10), cv=5):
    """
    Encuentra los mejores parámetros para la regresión logística utilizando ElasticNetCV.

    Parameters:
    - X: array-like, Características de entrada.
    - y: array-like, Etiquetas de clase.
    - alphas: array-like, Valores de alfa a probar.
    - l1_ratio: array-like, Valores de l1_ratio a probar.
    - cv: int, Número de divisiones en la validación cruzada.

    Returns:
    - best_alpha: float, Mejor valor de alfa.
    - best_l1_ratio: float, Mejor valor de l1_ratio.
    """
    # Inicializar el modelo ElasticNetCV
    elasticnet_cv = ElasticNetCV(alphas=alphas, l1_ratio=l1_ratio, cv=cv)

    # Ajustar el modelo a los datos
    elasticnet_cv.fit(X, y)

    # Obtener el mejor valor de alfa
    best_alpha = elasticnet_cv.alpha_

    # Obtener el mejor valor de l1_ratio
    best_l1_ratio = elasticnet_cv.l1_ratio_

    return best_alpha, best_l1_ratio


def train_elasticnet_classifier(X, y, alpha=1.0, l1_ratio=0.5):
    """
    Entrena un modelo de clasificación Elastic Net.

    Parameters:
    - X: array-like, Características de entrada.
    - y: array-like, Etiquetas de clase.
    - alpha: float, Parámetro de regularización (mayor valor significa una mayor regularización).
    - l1_ratio: float, Proporción de la regularización L1 (0 significa solo L2, 1 significa solo L1).

    Returns:
    - elasticnet_model: objeto, Modelo de clasificación Elastic Net entrenado.
    """
    elasticnet_model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    elasticnet_model.fit(X, y)
    return elasticnet_model

## Import dataset

In [ ]:
file = ('./adult.csv')
df = pd.read_csv(file, encoding='latin-1')

## Exploratory Data Analysis

In [ ]:
df.shape

Podemos ver que hay 32561 instancias y 15 atributos en el conjunto de datos.

In [ ]:
df.head()

In [ ]:
df.info()

### Encode `?` as `NaNs`

In [ ]:
df[df == '?'] = np.nan

In [ ]:
df.info()

Ahora, el resumen muestra que las variables `workclass`, `occupation` y `native.country` contienen valores faltantes. Todas estas variables son de tipo de datos categóricos. Entonces, imputaré los valores faltantes con el valor más frecuente: la moda.

In [ ]:
df.isnull().sum()

### Imputar valores faltantes con modo

In [ ]:
for col in ['workclass', 'occupation', 'native.country']:
    df[col].fillna(df[col].mode()[0], inplace=True)

### Verifique nuevamente si faltan valores

In [ ]:
df.isnull().sum()

Ahora podemos ver que no faltan valores en el conjunto de datos.

### Configuración del vector de características y la variable de destino

In [ ]:
X = df.drop(['income'], axis=1)
y = df['income'].replace({'<=50K':0,'>50K':1})

In [ ]:
X.head()

In [ ]:
y

## Dividir datos en conjuntos de prueba y entrenamiento separados

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

## Feature Engineering

### Codificar variables categóricas

In [ ]:
from sklearn import preprocessing

categorical = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])

## Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

In [ ]:
X_train.head()

## Modelo de regresión logística con todas las características.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

# print('Puntuación de precisión de regresión logística con todas las características: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

# # Evaluar el rendimiento del modelo
# accuracy = accuracy_score(y_test, y_pred)
# conf_matrix = confusion_matrix(y_test, y_pred)
# class_report = classification_report(y_test, y_pred)

# print("Precisión del modelo:", accuracy)
# # print("Matriz de confusión:\n", conf_matrix)
# print("Informe de clasificación:\n", class_report)

model_1 = calculate_metrics(y_test, logreg.predict(X_test), threshold=0.5)
model_1

## Regresión logística con PCA

La clase PCA de Scikit-Learn implementa el algoritmo PCA utilizando el siguiente código. Antes de profundizar, explicaré otro concepto importante llamado índice de varianza explicada.


### Relación de varianza explicada

Un dato muy útil es el **índice de varianza explicada** de cada componente principal. Está disponible a través de la variable `explained_variance_ratio_`. Indica la proporción de la varianza del conjunto de datos que se encuentra a lo largo del eje de cada componente principal.


In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
X_train = pca.fit_transform(X_train)

In [ ]:
# Obtener la proporción de varianza explicada por cada componente principal
explained_variance_ratio = pca.explained_variance_ratio_

# Calcular la varianza explicada acumulativa
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

# Imprimir la proporción de varianza explicada
print("Proporción de varianza explicada por cada componente principal:")
print(explained_variance_ratio)

# Graficar la proporción de varianza explicada y la varianza explicada acumulativa
plt.figure(figsize=(10, 6))

# Gráfico de barras de la proporción de varianza explicada
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, alpha=0.6, align='center', label='Varianza Explicada')

# Línea de la varianza explicada acumulativa
plt.step(range(1, len(cumulative_explained_variance) + 1), cumulative_explained_variance, where='mid', linestyle='--', label='Varianza Explicada Acumulativa')

# Configurar grillas
plt.grid(which='both', linestyle='--', linewidth=0.7)
plt.gca().yaxis.set_major_locator(plt.MultipleLocator(0.1))
plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(0.1))

# Método del codo
plt.xlabel('Componentes Principales')
plt.ylabel('Proporción de Varianza Explicada')
plt.title('Proporción de Varianza Explicada por los Componentes Principales')
plt.axhline(y=0.9, color='r', linestyle='-', label='Umbral de 90%')
plt.legend(loc='best')

plt.show()

### Comentario

- Podemos ver que aproximadamente el 97,25% de la varianza se explica por las 13 primeras variables.

- Sólo el 2,75% de la varianza se explica por la última variable. Entonces, podemos suponer que contiene poca información.

- Entonces se dejará, y entrenará el modelo nuevamente y con la precisión.



In [ ]:
# Cargas factoriales
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
loadings_df = pd.DataFrame(loadings, columns=[f'PC{i+1}' for i in range(len(loadings))], index=list(X.columns))

In [ ]:
loadings_df

In [ ]:
# Etiquetas para el eje x
etiquetas = X.columns

k = 7
n_rows = (loadings_df.shape[0] + 1) // k
fig, axes = plt.subplots(nrows=n_rows, ncols=k, figsize=(16, 10), sharey=True)

# Graficar las componentes principales
for i, (nombre_comp, comp) in enumerate(loadings_df.items()):
    row, col = i // k, i % k
    ax = axes[row, col]
    ax.bar(range(len(etiquetas)), comp, color=['teal', 'red'])
    ax.set_xticks(range(len(etiquetas)))
    ax.set_xticklabels(etiquetas, rotation=90)
    ax.axhline(y=0, color='black', linestyle='--')
    ax.set_title(nombre_comp)

# Ajustar espaciado y eliminar ejes vacíos
plt.subplots_adjust(hspace=0.5, wspace=0.3)
for ax in axes.flat[loadings_df.shape[0]:]:
    ax.axis('off')

plt.show()

In [ ]:
# Ajustar PCA con una cantidad específica de componentes
n_components = 12
pca = PCA(n_components=n_components)
X_pca_train = pca.fit_transform(X_train)
X_pca_test = pca.fit_transform(X_test)
feature_names = X.columns
# Proporción de varianza explicada
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

# Cargas factoriales
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

# Entrenar un modelo de regresión logística con los datos transformados
model = LogisticRegression(random_state=42)
model.fit(X_pca_train, y_train)

model_2 = calculate_metrics(y_test, model.predict(X_pca_test), threshold=0.5)
model_2

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

# Ajustar PCA con una cantidad específica de componentes
n_components = 10
pca = PCA(n_components=n_components)
X_pca_train = pca.fit_transform(X_train)
X_pca_test = pca.fit_transform(X_test)

# Entrenar un modelo de regresión logística con los datos transformados
model3 = LogisticRegression(random_state=42)
model3.fit(X_pca_train, y_train)

model_3 = calculate_metrics(y_test, model3.predict(X_pca_test), threshold=0.5)
model_3

In [ ]:
# Ajustar PCA con una cantidad específica de componentes

best_alpha, best_l1_ratio = find_best_elasticnet_params(X_train, y_train)
logistic_model = train_elasticnet_classifier(X_train, y_train, alpha=best_alpha, l1_ratio=best_l1_ratio)
mt_en_1 = calculate_metrics(y_test, logistic_model.predict(X_test), threshold=0.5)
mt_en_1

In [ ]:
# Ajustar PCA con una cantidad específica de componentes
n_components = 12
pca = PCA(n_components=n_components)
X_pca_train = pca.fit_transform(X_train)
X_pca_test = pca.fit_transform(X_test)

best_alpha, best_l1_ratio = find_best_elasticnet_params(X_pca_train, y_train)
logistic_model = train_elasticnet_classifier(X_pca_train, y_train, alpha=best_alpha, l1_ratio=best_l1_ratio)
mt_en_2 = calculate_metrics(y_test, logistic_model.predict(X_pca_test), threshold=0.5)
mt_en_2

In [ ]:
# Ajustar PCA con una cantidad específica de componentes
n_components = 10
pca = PCA(n_components=n_components)
X_pca_train = pca.fit_transform(X_train)
X_pca_test = pca.fit_transform(X_test)

best_alpha, best_l1_ratio = find_best_elasticnet_params(X_pca_train, y_train)
logistic_model = train_elasticnet_classifier(X_pca_train, y_train, alpha=best_alpha, l1_ratio=best_l1_ratio)
mt_en_3 = calculate_metrics(y_test, logistic_model.predict(X_pca_test), threshold=0.5)
mt_en_3

In [ ]:
resultados = pd.concat([pd.DataFrame([model_1]),
           pd.DataFrame([model_2]),
           pd.DataFrame([model_3]),
           pd.DataFrame([mt_en_1]),
           pd.DataFrame([mt_en_2]),
           pd.DataFrame([mt_en_3]),],axis=0,).T

resultados.columns = ['Logit_o','Logit_12PC','Logit_10PC','EN_o','EN_12PC','EN_10PC']

In [ ]:
resultados